Before you turn this problem in, make sure everything runs as expected. First, **restart the kernel** (in the menubar, select Kernel$\rightarrow$Restart) and then **run all cells** (in the menubar, select Cell$\rightarrow$Run All).

Make sure you fill in any place that says `YOUR CODE HERE` or "YOUR ANSWER HERE", as well as your name and collaborators below:

In [ ]:
NAME = "Anamarija Hauptman"
COLLABORATORS = ""

In [ ]:
import numpy as np
import scipy.ndimage as ndi
from scipy.interpolate import interpn
import PIL.Image as Image
import rvlib as rv
import time
import matplotlib.pyplot as plt

---

# Domača naloga 3

Pri vseh nalogah uporabite sliko na naslovu `"./data/slika.JPG"`.

1. Uporabite funkcijo `convolve()` v Python knjižnici `scipy.ndimage` za izračun 2D diskretne konvolucije na sivinski sliki z jedrom `iKernel = numpy.ones([11,11])/121`. Prikažite dobljeno sliko in izpišite čas trajanja izračuna v milisekundah. Trajanje izračunajte z uporabo funkcije `clock()` v knjižnjici `time` in ga nato ustrezno pretvorite.

In [ ]:
img = rv.loadImage('./data/slika.JPG')
iKernel = np.ones([11, 11]) / 121
gray_img = rv.colorToGray(img)

t1 = time.perf_counter_ns()
convolve_img = ndi.convolve(gray_img, iKernel)
t2 = time.perf_counter_ns()

print('cas trajanja izpisa:', (t2 - t1) / 10e6, 'milisekund')

rv.showImage(convolve_img, iTitle='2D diskretna konvolucija z jedrom velikosti 11x11')

In [ ]:
iKernel = np.ones([11, 11]) / 121

2. V funkciji diskretne 2D konvolucije napisani na vajah smo privzeli, da so sivinske vrednosti točk zunaj slike enake 0. Pri uporabi funkcije z večjimi konvolucijskimi jedri zaradi tega na izhodnih slikah dobimo popačen rob. Obstajajo različni načini za odpravo teh popačenj. Opišite in analizirajte vse možne načine, ki jih omogoča funkcija `convolve()` v Python knjižnici `scipy.ndimage` (parameter mode), ter prikažite rezultate. Vizualno ocenite kako se razlikujejo rezultati različnih načinov pri manjših in večjih velikostih jedra za povprečenje (npr. 5 in 21).

In [ ]:
img = rv.loadImage('./data/slika.JPG')
iKernel1 = np.ones([5, 5]) / 25
iKernel2 = np.ones([21, 21]) / 441
gray_img = rv.colorToGray(img)

2. 1. Opišite in analizirajte način "reflect" ter prikažite na ta način izračunano sliko

`‘reflect’`: (d c b a | a b c d | d c b a) Zrcaljenje čez rob

In [ ]:
convolve_img1 = ndi.convolve(gray_img, iKernel1, mode='reflect')
convolve_img2 = ndi.convolve(gray_img, iKernel2, mode='reflect')

fig, axs = plt.subplots(1, 2, figsize=(10, 8))
axs[0].set_title('Kernel size = 5')
axs[0].imshow(convolve_img1, cmap='gray')
axs[1].set_title('Kernel size = 21')
axs[1].imshow(convolve_img2, cmap='gray')
plt.tight_layout()
plt.show()

2. 2. Opišite in analizirajte način "constant" ter prikažite na ta način izračunano sliko

`constant`(k k k k | a b c d | k k k k)
Sivinske vrednosti tock izven slike so enake konstanti `cval`. Prednastavljena vrednost je enaka 0.

Default je enak naši implementirani verziji -- rob je popačen.

In [ ]:
convolve_img1 = ndi.convolve(gray_img, iKernel1, mode='constant', cval=0)
convolve_img2 = ndi.convolve(gray_img, iKernel2, mode='constant', cval=0)

fig, axs = plt.subplots(1, 2, figsize=(10, 8))
axs[0].set_title('Kernel size = 5')
axs[0].imshow(convolve_img1, cmap='gray')
axs[1].set_title('Kernel size = 21')
axs[1].imshow(convolve_img2, cmap='gray')
plt.tight_layout()
plt.show()

2. 3. Opišite in analizirajte način "nearest" ter prikažite na ta način izračunano sliko

`‘nearest’` (a a a a | a b c d | d d d d)
Sivinske vrednosti točk izven slike so enake robnim pikslom.

In [ ]:
convolve_img1 = ndi.convolve(gray_img, iKernel1, mode='nearest')
convolve_img2 = ndi.convolve(gray_img, iKernel2, mode='nearest')

fig, axs = plt.subplots(1, 2, figsize=(10, 8))
axs[0].set_title('Kernel size = 5')
axs[0].imshow(convolve_img1, cmap='gray')
axs[1].set_title('Kernel size = 21')
axs[1].imshow(convolve_img2, cmap='gray')
plt.tight_layout()
plt.show()

2. 4. Opišite in analizirajte način "mirror" ter prikažite na ta način izračunano sliko

`‘mirror’` (d c b | a b c d | c b a)
Sivinske vrednosti točk izven slike so zrcaljene čez rob, kjer zadnjipiksel slike predstavlja center.


In [ ]:
convolve_img1 = ndi.convolve(gray_img, iKernel1, mode='mirror')
convolve_img2 = ndi.convolve(gray_img, iKernel2, mode='mirror')

fig, axs = plt.subplots(1, 2, figsize=(10, 8))
axs[0].set_title('Kernel size = 5')
axs[0].imshow(convolve_img1, cmap='gray')
axs[1].set_title('Kernel size = 21')
axs[1].imshow(convolve_img2, cmap='gray')
plt.tight_layout()
plt.show()

2. 5. Opišite in analizirajte način "wrap" ter prikažite na ta način izračunano sliko

`‘wrap’` (a b c d | a b c d | a b c d)

The input is extended by wrapping around to the opposite edge.

In [ ]:
convolve_img1 = ndi.convolve(gray_img, iKernel1, mode='wrap')
convolve_img2 = ndi.convolve(gray_img, iKernel2, mode='wrap')

fig, axs = plt.subplots(1, 2, figsize=(10, 8))
axs[0].set_title('Kernel size = 5')
axs[0].imshow(convolve_img1, cmap='gray')
axs[1].set_title('Kernel size = 21')
axs[1].imshow(convolve_img2, cmap='gray')
plt.tight_layout()
plt.show()

3. Napišite funkcijo `discreteConvolutionColorImage`, ki bo omogočala konvolucijo barvnih slik s poljubnim jedrom. Preizkusite delovanje na barvni sliki z jedrom `iKernel = numpy.ones([11,11])/121` in prikažite sliko.

In [ ]:
# Implementirajte funkcijo
def discreteConvolutionColorImage(iImage, iKernel, iMode = 'mirror'):
    #Your code here
    if iImage.ndim == 3 and iImage.shape[0] == 3:
        iImage = np.transpose(iImage, [1, 2, 0])
    oImage = np.zeros_like(iImage)
    for i in range(iImage.shape[2]):
        oImage[:, :, i] = ndi.convolve(iImage[:, :, i], iKernel, mode=iMode)
    return oImage
img = rv.loadImage('./data/slika.JPG')
iKernel = np.ones([11,11]) / 121
img_cc = discreteConvolutionColorImage(img, iKernel)
#plt.figure()
#plt.imshow(img_cc)
#plt.title('Konvolucija barvne slike z jedrom velikosti 11x11')
#plt.show()
rv.showImage(img_cc, iTitle='Konvolucija barvne slike z jedrom velikosti 11x11')

In [ ]:
from nose.tools import assert_equal, assert_almost_equal
test_image = np.zeros((100, 100, 3), dtype='uint8')
test_image[46:49, 46:49, 1] = 255
test_image[46:49, 46:49, 0] = 25
test_kernel = np.ones((3, 3)) / 9
result = discreteConvolutionColorImage(test_image, test_kernel)

expected_result = np.zeros_like(test_image)
expected_result[45:50, 45:50, 0] = np.array(
    [[2,  5,  8,  5, 2],
     [5, 11, 16, 11, 5],
     [8, 16, 25, 16, 8],
     [5, 11, 16, 11, 5],
     [2,  5,  8,  5, 2]])
expected_result[45:50, 45:50, 1] = np.array(
    [[28,  56,  85,  56,  28],
     [56, 113, 170, 113,  56],
     [85, 170, 255, 170,  85],
     [56, 113, 170, 113,  56],
     [28,  56,  85,  56,  28]])
assert_equal(np.sum(result - expected_result), 0)

In [ ]:
# Prikažite sliko


4. 1. Napišite funkcijo za izračun konvolucijskega jedra v obliki 2D simetrične Gaussove funkcije:
```python
    def discreteGaussian2D( iSigma ):
        return oKernel
```
 kjer je `iSigma` standardna deviacija $\sigma$ v 2D simetrični Gaussovi funkciji, ki je definirana kot: 
   $k(u, v) = (2\pi)^{-1}\sigma^{-2}e^{-(u^2 + v^2)/{2\sigma^2}}$ Velikost konvolucijskega jedra $U \times V$ nastavite glede na vrednost `iSigma` kot $U, V = 2 \cdot (3 \sigma) + 1$, kjer tako pridobljene vrednosti $U$ in $V$ zaokrožite s funkcijo `numpy.ceil`, vrednost $k(0, 0)$ pa naj bo v središčnem elementu izhodne 2D matrike `oKernel`. Zagotovite, da bo vsota vseh elementov `oKernel` enaka 1. 

In [ ]:
# Implementirajte funkcijo za izračun konvolucijskega jedra
def discreteGaussian2D(iSigma):
    size = int(2 * np.ceil(3 * iSigma) + 1)
    center = size // 2
    oKernel = np.zeros((size, size))
    for u in range(size):
        for v in range(size):
            distance = ((u - center) ** 2 + (v - center) ** 2) / (2 * iSigma ** 2)
            oKernel[u, v] = np.exp(-distance) / (2 * np.pi * iSigma ** 2)
    oKernel /= np.sum(oKernel)
    return oKernel

In [ ]:
from nose.tools import assert_equal, assert_almost_equal
k1 = discreteGaussian2D(1)
k3 = discreteGaussian2D(3)
assert_equal(k1.shape, (7, 7))
assert_equal(k3.shape, (19, 19))
assert_almost_equal(np.sum(k1), 1, places=3)
assert_almost_equal(np.sum(k3), 1, places=3)
assert_almost_equal(k1[3, 3], 0.1592, places=3)
assert_almost_equal(k3[9, 9], 0.017735, places=3)

4. 2. Določite konvolucijski jedri za $\sigma = 0.1$ in $3$. Z jedroma izvedite diskretno 2D konvolucijo na barvni sliki in ju prikažite. Obrazložite vpliv vrednosti $\sigma$.

In [ ]:
kernel01 = discreteGaussian2D(0.1)
kernel3 = discreteGaussian2D(3)

img = rv.loadImage('./data/slika.JPG')
img_con_01 = discreteConvolutionColorImage(img, kernel01)
img_con_3 = discreteConvolutionColorImage(img, kernel3)

if img.ndim == 3 and img.shape[0] == 3:
    img = np.transpose(img, [1, 2, 0])
rv.showImage(img, iTitle='Originalna slika')

In [ ]:
rv.showImage(img_con_01, iTitle='sigma = 0.1')

In [ ]:
rv.showImage(img_con_3, iTitle='sigma = 3')

Za majhno vrednost $\sigma$ (npr. $\sigma = 0.1$) je vpliv glajenja majhen, saj je utež elementa $(0,0)$ skoraj 1. Večja kot je vrednost $\sigma$, večja je torej disperzija in večje so pripisane uteži okoliškim elementom $(u,v), u,v\neq 0$. Ker sta tudi vrednosti $U$ in $V$ linearno odvisni od vrednosti $\sigma$ bo večja tudi velikost filtra. 

5. Ostrenje slike je analogno prostorskemu odvajanju sivinskih vrednosti. Ostrenje slik z drugim odvodom izvedemo z uporabo Laplaceovega jedra tako, da vhodni sliki $g(x, y)$ odštejemo uteženo sliko drugega odvoda slike: 
 
 $$f(x, y) = g(x, y) - c \Delta g(x, y)$$

 kjer konstanta $c$ določa stopnjo ostrenja. 
 1. Implementirajte funkcijo, ki izostri sivinsko sliko. Poskrbite da, po odštevanju oz. prištevanju slike drugega odvoda $g(x, y)$ zagotovite, da bodo vrednosti 8-bitne sivinske slike na območju od $[0, 255]$, pri čemer vrednosti manjše od 0 oz. vrednosti večje od 255 ustrezno zaokrožite. 
 2. Izostrite sivinsko sliko z vrednostmi $c = 0, 0.5, 1, 2$ in slike prikažite. Preizkusite različne vrednosti $c$ in obrazložite njihov vpliv na rezultirajočo sivinsko sliko. Če vam čas dopušča preizkusite ostrenje še na barvni sliki.

<img src="./images/RV_4_SLO_LaplaceJedro.png" alt="Laplace" width="150"/>

In [ ]:
# 5. A. Implementirajte funkcijo za ostrenje sivinskih slik
def sharpenImage2D(iImage, c):
    Laplace = np.array([[0, 1, 0], [1, -4, 1], [0, 1, 0]])
    deltaG = ndi.convolve(iImage, Laplace).astype('uint8')
    oImage = iImage - c * deltaG
    oImage = np.clip(oImage, 0, 255)
    return oImage.astype('uint8')

In [ ]:
def sharpenImageColor(iImage, c):
    oImage = []
    if iImage.ndim == 3 and iImage.shape[0] == 3:
        iImage = np.transpose(iImage, [1, 2, 0])
    for i in range(iImage.shape[2]):
        oImage.append(sharpenImage2D(iImage[:, :, i], c))
    oImage = np.array(oImage).astype('uint8')
    return oImage

In [ ]:
img_gray = rv.colorToGray(img)

rv.showImage(sharpenImage2D(img_gray, 0), iTitle='c = 0')

In [ ]:
img_gray = rv.colorToGray(img)

rv.showImage(sharpenImage2D(img_gray, 0.5), iTitle='c = 0.5')

In [ ]:
img_gray = rv.colorToGray(img)

rv.showImage(sharpenImage2D(img_gray, 1), iTitle='c = 1')

In [ ]:
img_gray = rv.colorToGray(img)

rv.showImage(sharpenImage2D(img_gray, 2), iTitle='c = 2')

In [ ]:
img = rv.loadImage('./data/slika.JPG')
img_sharp = sharpenImageColor(img, 1)
if img_sharp.ndim == 3 and img_sharp.shape[0] == 3:
    img_sharp = np.transpose(img_sharp, [1, 2, 0])
rv.showImage(img_sharp)

5. 3. Opišite vpliv faktorja $c$ na izostreno sliko:

Pocečevanje vrednosti nekoliko poudari robove. Homogeni deli sivinske slike ostanejo približno enaki. Izjemno se poveča število nasičenih pikslov (šum sol/poper): z naraščanjem vrednosti c postane problem še bolj izrazit.

6. Napišite funkcijo `interpolateColorImage`, ki bo omogočala uporabo interpolacije ničtega (`method='nearest'`) in prvega (`method='linear'`) reda na barvnih slikah. Uporabite funkcijo `interpn()` iz knjižnice `scipy.interpolate`. Preizkusite delovanje funkcije tako, da z ničtim in prvim redom prevzorčite del barvne slike od točke $(50, 100)$ do točke $(200, 250)$ s petkrat bolj gosto mrežo točk. Prikažite originalni del slike in prevzorčeno sliko.

In [ ]:
# Implementirajte funkcijo za interpolacijo barvne slike
def interpolateColorImage(iImage, iCoorX, iCoorY, method='linear'):
    #Your code here
    if iImage.ndim == 3 and iImage.shape[0] == 3:
        iImage = np.transpose(iImage, [1, 2, 0])
    oImage = []
    for channel in range(iImage.shape[2]):
        interpolated_channel = interpn((iCoorX, iCoorY), iImage[:, :, channel], (iCoorX, iCoorY), method=method)
        oImage.append(interpolated_channel)
    oImage = np.stack(oImage, axis=2)
    return oImage

img = rv.loadImage('./data/slika.JPG')
if img.ndim == 3 and img.shape[0] == 3:
    img = np.transpose(img, [1, 2, 0])
img = img[100:250, 50:200, :]
rv.showImage(img, iTitle='Originalni del slike')

In [ ]:
from nose.tools import assert_equal, assert_almost_equal
test_image = np.zeros((100, 100, 3), dtype='uint8')
test_image[40:50, 40:50, 0] = 100
iCoorX, iCoorY = np.meshgrid(np.arange(40, 49, 1/5), np.arange(40, 49, 1/5), indexing='xy')
result = interpolateColorImage(test_image, iCoorX, iCoorY, 'nearest')
assert_equal(result.shape, (45, 45, 3))
assert(np.all(result[:, :, 0] == 100))
assert(np.all(result[:, :, 1] == 0))
assert(np.all(result[:, :, 2] == 0))
result = interpolateColorImage(test_image, iCoorX, iCoorY, 'linear')
assert_equal(result.shape, (45, 45, 3))
assert(np.all(result[:, :, 0] == 100))
assert(np.all(result[:, :, 1] == 0))
assert(np.all(result[:, :, 2] == 0))

In [ ]:
rangeX = np.linspace(50, 200, img.shape[0])
rangeY = np.linspace(100, 250, img.shape[1])
meshgridX, meshgridY = np.meshgrid(rangeX, rangeY)
img_nearest = interpolateColorImage(img, meshgridX, meshgridY, method='nearest')
rv.showImage(img_nearest, iTitle='Interpolacija ničtega reda')

In [ ]:
img_linear = interpolateColorImage(img, meshgrid_x, meshgrid_y, method='linear')
rv.showImage(img_linear, iTitle='Interpolacija prvega reda')

7. Prilagodite funkcijo za piramidno decimacijo tako, da bo omogočala decimiranje barvnih slik in preizkusite delovanje na barvni sliki. Pomanjšajte sliko za 2-krat in 4-krat.

In [ ]:
def zeroPad(image, m, n):
    shape = image.shape + np.array([2*m, 2*n])
    padded_image = np.zeros(shape)
    padded_image[m:shape[0] - m, n:shape[1] - n] = image
    return padded_image

def discreteConvolution2D(iImage, iKerne):
    n = np.array(iKerne.shape[0] // 2).astype('uint8')
    m = np.array(iKerne.shape[1] // 2).astype('uint8')
    Y, X = iImage.shape
    iImage = zeroPad(iImage, m, n)
    oImage = np.zeros(iImage.shape)
    for i in range(n, Y+n):
        for j in range(m, X+m):
            subImage = iImage[i-n:i+n+1, j-m:j+m+1]
            oImage[i, j] = np.sum(subImage * iKerne)
    oImage = oImage[n:Y+n, m:X+m]
    return oImage

def decimateImage2D(iImage, iLevel, iKernel=np.array([[1/16,1/8, 1/16],[1/8, 1/4, 1/8],[1/16, 1/8, 1/16]])):
    if iLevel <= 0:
        return iImage
    else:
        oImage = iImage
        dy, dx = oImage.shape
        X, Y = np.meshgrid(np.arange(0, dx, 2), np.arange(0, dy, 2), indexing='xy')
        oImage = discreteConvolution2D(oImage, iKernel)
        oImage = oImage[Y, X]
        return decimateImage2D(oImage, iLevel-1, iKernel)
    
def decimateColorImage(iImage, iLevel, iKernel=np.array([[1/16,1/8, 1/16],[1/8, 1/4, 1/8],[1/16, 1/8, 1/16]])):
    #Your code here
    if iImage.ndim == 3 and iImage.shape[0] == 3:
        iImage = np.transpose(iImage, [1, 2, 0])
    #oImage = np.zeros_like(iImage)
    oImage = []
    for i in range(iImage.shape[2]):
        #oImage[:, :, i] = decimateImage2D(iImage[:, :, i], iLevel, iKernel)
        oImage.append(decimateImage2D(iImage[:, :, i], iLevel, iKernel))
    oImage = np.array(oImage).astype('uint8')
    return oImage

In [ ]:
img = rv.loadImage('./data/slika.JPG')
if img.ndim == 3 and img.shape[0] == 3:
    img = np.transpose(img, [1, 2, 0])

img_dec2 = decimateColorImage(img, 2)
if img_dec2.ndim == 3 and img_dec2.shape[0] == 3:
    img_dec2 = np.transpose(img_dec2, [1, 2, 0])
rv.showImage(img_dec2)

In [ ]:
img_dec4 = decimateColorImage(img, 4)
if img_dec4.ndim == 3 and img_dec4.shape[0] == 3:
    img_dec4 = np.transpose(img_dec4, [1, 2, 0])
rv.showImage(img_dec4)